In [1]:
push!(LOAD_PATH, "../")
using Integrator
using SpectralGrid
using CubicBSpline
using Chebyshev
using Fourier
using NumericalModels
using Parameters
using CSV
using DataFrames

In [2]:
nodes = 100
lpoints = 0
blpoints = 0
for r = 1:(nodes*3)
    lpoints += 4 + 4*r
    blpoints += 1 + 2*r
end
model = ModelParameters(
    ts = 1.0,
    integration_time = 3600.0,
    output_interval = 3600.0,
    equation_set = "LinearShallowWaterRL",
    initial_conditions = "gauss_RL_ics.csv",
    output_dir = "./linearSW_output/",
    grid_params = GridParameters(xmin = 0.0,
        xmax = 3.0e5,
        num_nodes = nodes,
        rDim = nodes*3,
        b_rDim = nodes+3,
        BCL = Dict(
            "h" => CubicBSpline.R0, 
            "u" => CubicBSpline.R1T0, 
            "v" => CubicBSpline.R1T0,
            "ub" => CubicBSpline.R1T0, 
            "vb" => CubicBSpline.R1T0,
            "wb" => CubicBSpline.R1T0),
        BCR = Dict(
            "h" => CubicBSpline.R0, 
            "u" => CubicBSpline.R0, 
            "v" => CubicBSpline.R0,
            "ub" => CubicBSpline.R0, 
            "vb" => CubicBSpline.R0,
            "wb" => CubicBSpline.R0),
        lDim = lpoints,
        b_lDim = blpoints,
        vars = Dict(
            "h" => 1, 
            "u" => 2, 
            "v" => 3,
            "ub" => 4, 
            "vb" => 5,
            "wb" => 6)))

ModelParameters
  ts: Float64 1.0
  integration_time: Float64 3600.0
  output_interval: Float64 3600.0
  equation_set: String "LinearShallowWaterRL"
  initial_conditions: String "gauss_RL_ics.csv"
  output_dir: String "./linearSW_output/"
  grid_params: GridParameters


In [3]:
grid = createGrid(model.grid_params);

In [4]:
gridpoints = getGridpoints(grid)
cartesian = getCartesianGridpoints(grid);

In [5]:
Rmax = 50000.0
Vmax = 50.0
V0 = Vmax / Rmax
f = 0.0
#V = 0.0
#U = -10.0
r_1 = 0.0
h = 0.0
v = 0.0
dhdr_1 = 0.0
for i = 1:grid.params.lDim
    r_m = gridpoints[i,1]
    l_m = gridpoints[i,2]
    x_m = cartesian[i,1]
    y_m = cartesian[i,2]
    if (r_m < Rmax)
        v = V0 * r_m
    else
        v = Rmax * Rmax * V0 / r_m
    end
    if r_m > r_1
        dhdr = ((f * v) + (v * v / r_m)) / 9.81
        h = h + (r_m - r_1) * 0.5 * (dhdr + dhdr_1)
        dhdr_1 = dhdr
        r_1 = r_m
    end
    grid.physical[i,1,1] = h
    # Gaussian blob test
    #grid.physical[i,1,1] = 100.0*exp(-((x_m-5.0e4)^2 / (2 * 1.0e4^2)) - 
    #    ((y_m)^2 / (2 * 1.0e4^2))) + 100.0*exp(-((x_m+5.0e4)^2 / (2 * 1.0e4^2)) - 
    #    ((y_m)^2 / (2 * 1.0e4^2)))
    #grid.physical[i,2,1] = (U * cos(l_m)) + (V * sin(l_m))
    grid.physical[i,2,1] = 0.0
    grid.physical[i,3,1] = v
    grid.physical[i,4,1] = 0.0
    grid.physical[i,5,1] = v
    grid.physical[i,6,1] = 0.0
end

In [6]:
grid.physical[:,1,1] = grid.physical[:,1,1] .- 251.25623619652322

181800-element Vector{Float64}:
 -251.25040974451437
 -251.25040974451437
 -251.25040974451437
 -251.25040974451437
 -251.25040974451437
 -251.25040974451437
 -251.25040974451437
 -251.25040974451437
 -251.14155729744064
 -251.14155729744064
 -251.14155729744064
 -251.14155729744064
 -251.14155729744064
    ⋮
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0

In [7]:
ics = DataFrame(r=gridpoints[:,1],
    l=gridpoints[:,2], 
    h=grid.physical[:,1,1], 
    u=grid.physical[:,2,1], 
    v=grid.physical[:,3,1],
    ub = grid.physical[:,4,1],
    vb = grid.physical[:,5,1],
    wb = grid.physical[:,6,1]) 

,r,l,h,u,v,ub,vb,wb
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,338.105,0.0,-251.25,0.0,0.338105,0.0,0.338105,0.0
2,338.105,0.785398,-251.25,0.0,0.338105,0.0,0.338105,0.0
3,338.105,1.5708,-251.25,0.0,0.338105,0.0,0.338105,0.0
4,338.105,2.35619,-251.25,0.0,0.338105,0.0,0.338105,0.0
5,338.105,3.14159,-251.25,0.0,0.338105,0.0,0.338105,0.0
6,338.105,3.92699,-251.25,0.0,0.338105,0.0,0.338105,0.0
7,338.105,4.71239,-251.25,0.0,0.338105,0.0,0.338105,0.0
8,338.105,5.49779,-251.25,0.0,0.338105,0.0,0.338105,0.0
9,1500.0,0.261799,-251.142,0.0,1.5,0.0,1.5,0.0


In [9]:
CSV.write("SWslab_rankine_ics.csv",ics)

"SWslab_rankine_ics.csv"